In [3]:
import numpy as np
import matplotlib.pyplot as plt
from CADMium import Kohnsham, Pssolver, Psgrid, Partition, Inverter


---
### Perform PDFT Calculation. 
Currently the method used is "OrbitalInvert". 
But original code may have used "WuYang". 
Code should run as it is but for idential calculations increase to grid size to: [7,12,12]

In [34]:
a = 5.122/2
#Nuclear charge for fragments A and B
Za, Zb = 3,3
#Set polarization 1-Unpolarized, 2-Polarized
pol = 2
#Fragment a electrons [alpha, beta]
Nmo_a = [[2,1]] #Number of molecular orbitals to calculate
N_a   = [[2,1]]
#Ensemble mix
nu_a = 1
#Fragment b electrons
Nmo_b = [[2,1]]
N_b   = [[2,1]]
#Ensemble mix
nu_b = 1

#Molecular elctron configuration
Nmo_m = [[3,3]]
N_m   = [[3,3]]

#Grid Options
NP = 7 #Number of points per block
NM =  [6,6] #Number of blocks [angular, radial]
L = np.arccosh(15./a) #Maximum radial coordinate value
loc = np.array(range(-4,5)) #Non inclusive on upper bound
grid = Psgrid(NP, NM, a, L, loc)
grid.initialize()



#Initialize required objects. And make calculation in isolated fragments for initial guess. 

part = Partition(grid, Za, Zb, pol, Nmo_a, N_a, nu_a, Nmo_b, N_b, nu_b, { "kinetic_part_type" : 'inversion',
                                                                          "vp_calc_type"      : "component",
                                                                          "ab_sym"            : True,
                                                                          "ens_spin_sym"      : True,})

#Setup inverter object
mol_solver = Pssolver(grid, Nmo_m, N_m)
part.inverter = Inverter(grid, mol_solver, {"invert_type"     : "orbitalinvert",
                                            "tol_invert"      : 1e-10,
                                            "max_iter_invert" : 100,
                                            "disp"            : False,
                                            "ab_sym"          : True,
                                            "ens_spin_sym"    : True,})

part.optPartition.isolated = True
part.scf({"disp"  : False,
          "alpha" : [0.6],
          "e_tol" : 1e-12})

D0_frag_a = part.KSa.n.copy()
D0_frag_b = part.KSa.n.copy()


#Turn off iterative linear solver for each solver
part.KSa.solver[0][0].optSolver.iter_lin_solver = False
part.KSa.solver[0][1].optSolver.iter_lin_solver = False


part.optPartition.isolated   = False

part.scf({"disp"       : True,
          "alpha"      : [0.6],
          "max_iter"   : 200,
          "e_tol"      : 1e-9,
          "continuing" : True})

#Store full densities under the presence of vp.
Dvp_frag_a = part.KSa.n.copy()
Dvp_frag_b = part.KSb.n.copy()

                Total Energy ( a.u.)                               Inversion                

                __________________                ____________________________________     

Iteration         A              B                  iters      optimality        res       

___________________________________________________________________________________________ 

    1            -7.32464        -7.32464             9       +4.669e-11      +1.000e+00
    2            -7.32866        -7.32866            10       +2.640e-11      +4.569e-02
    3            -7.34060        -7.34060             8       +2.320e-11      +2.900e-02
    4            -7.34060        -7.34060             9       +4.822e-15      +2.343e-02
    5            -7.33583        -7.33583             8       +5.505e-14      +9.256e-03
    6            -7.33822        -7.33822             6       +5.684e-11      +6.618e-03
    7            -7.33610        -7.33610             6       +2.891e-12      +7.186e-03
   

---
### Generate Figure 9. Parititon Potential. 

In [64]:
full, x,y = grid.plotter(part.V.vp[:,0])
fig, ax = plt.subplots(dpi=300)

plot = ax.contourf(x,y,full, levels=22, cmap="plasma")

ax.set_aspect('equal')
ax.set_xlim([-7,7])
ax.set_ylim([-7,7])

ax.scatter(5.122/2, 0, color='white', s=20)
ax.scatter(-5.122/2, 0, color='white', s=15)

ax.axis('off')

# fig.colorbar(plot)
# plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-7.0, 7.0, -7.0, 7.0)

---
### Generate Figure 9. Difference between Fragment Density and Isolated Atomic Density. 

In [36]:
D_grid, x, y = grid.plotter(D0_frag_a[:,0])
D_vp_grid, _, _ = grid.plotter(Dvp_frag_a[:,0])

fig, ax = plt.subplots(dpi=150)
plot = plt.contourf(x,y, D_vp_grid - D_grid, levels=100, cmap="jet")

ax.set_aspect('equal')
ax.set_xlim([-5,5])
ax.set_ylim([-5,5])

fig.colorbar(plot)
# plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

---
### Generate Figure 11. Components of the Partition Potential

In [37]:
x_axis, vp      = grid.axis_plot(part.V.vp[:,0])
x_axis, vp_kin  = grid.axis_plot(part.V.vp_kin[:,0])
x_axis, vp_xc   = grid.axis_plot(part.V.vp_x[:,0] + part.V.vp_c[:,0] )
x_axis, vp_hext = grid.axis_plot( part.V.vp_h[:,0] + part.V.vp_pot[:,0])

fig, ax = plt.subplots(dpi=150)

ax.plot(x_axis, vp, label='Total')
ax.plot(x_axis, vp_kin, label='Kinetic')
ax.plot(x_axis, vp_xc, label='XC')
ax.plot(x_axis, vp_hext, label="H + Vext")

ax.set_xlim(0,7)
ax.set_ylim(-0.12, 0.12)

ax.legend()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

---
### Generate Table 9. Energies and Components of Ep, in atomic Units

In [6]:
values = {}
for i in part.E.__dict__:
    if i.startswith("__") is False:
        values.update({i : getattr(part.E, i)})
values

{'Ea': -7.337784849443326,
 'Eb': -7.337784849443326,
 'Ef': -14.675569698886653,
 'Tsf': 14.515977603251898,
 'Eksf': array([[-3.87274135, -3.62105118]]),
 'Enucf': -33.88821469087026,
 'Exf': -3.041843485917988,
 'Ecf': -0.3009939391861839,
 'Ehf': 8.039504813835878,
 'Vhxcf': 11.684634289090768,
 'Ep': -1.806918223995157,
 'Ep_pot': -3.6782118538543833,
 'Ep_kin': 0.004784336086718355,
 'Ep_hxc': 1.866509293772508,
 'Et': -16.48248792288181,
 'Vnn': 1.757126122608356,
 'E': -14.725361800273452,
 'evals_a': array([], dtype=float64),
 'evals_b': array([], dtype=float64),
 'Ep_h': 1.8866106216767342,
 'Ep_x': 0.007092235124382729,
 'Ep_c': -0.027193563028608814}

In [63]:
%matplotlib widget

fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, dpi=300)
surf = ax.plot_surface(x, y, full, cmap="plasma", alpha=1.0, 
                       linewidth=20, antialiased=True)

ax.grid(False)
ax.set_axis_off()
ax.dist = 6
ax.set_facecolor("white")

fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …